# This notebook demonstrates how to implement and train a simple linear regression model using PyTorch.
# It covers data generation, model definition, loss function and optimizer setup, training loop, and evaluation of results.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. Prepare Data
# Generate some synthetic data for linear regression: y = 2*x + 1 + noise
X = torch.randn(100, 1) * 10  # 100 data points, 1 feature
y = 2 * X + 1 + torch.randn(100, 1) * 2  # True relationship + noise

# 2. Define the Model
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(in_features=1, out_features=1) # One input feature, one output

    def forward(self, x):
        return self.linear(x)

model = LinearRegression()

# 3. Define Loss Function and Optimizer
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.SGD(model.parameters(), lr=0.01) # Stochastic Gradient Descent with learning rate

# 4. Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass: Compute predicted y by passing x to the model
    outputs = model(X)
    loss = criterion(outputs, y)

    # Backward and optimize: Zero gradients, perform backpropagation, update weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 5. Make Predictions (Optional)
predicted = model(X).detach().numpy() # Detach from computation graph and convert to NumPy
print("\nFirst 5 true values:", y[:5].numpy().flatten())
print("First 5 predicted values:", predicted[:5].flatten())

# Print learned parameters
print("\nLearned Weight:", model.linear.weight.item())
print("Learned Bias:", model.linear.bias.item())


ModuleNotFoundError: No module named 'torch'

## Problem Summary:
We are tasked with predicting daily sales of FMCG products from a warehouse, using both historical sales data and local weather (temperature) information.

## Solution Overview:
We will use a linear regression model to learn the relationship between temperature and product sales. The model will be trained on historical data, and then used to predict future sales based on temperature forecasts.

## Next Steps:
### 1. Load and explore the sales and temperature datasets.
### 2. Prepare the data for modeling (e.g., merging, cleaning, feature selection).
### 3. Train the linear regression model.
### 4. Evaluate model performance and make predictions.
### 5. Save and reload the model for future use.


1. Define a use case and dataset associated with it
2. Use the Linear Regression Model for the use case
3. Ability to save a model as a checkpoint
4. Retrieve the model and retrain with new training dataset
5. Validation and model performance
6. Scoring pipeline using quick API design
7. Key learnings and discussion

You are managing a FMCG product warehouse and you need to manage the products sold from the warehouse along with dependant weather patterns around the warehouse location.

You are given a data about the sales daily and you are given the temperature of the town where the warehouse is located.


In [ ]:
# Prepare the data
import pandas as pd
import numpy as np

sales_data = pd.read_csv('ohio_daily_vegetable_sales.csv')

In [ ]:
temp_data = pd.read_csv('ohio_daily_avg_temperature.csv')

In [ ]:
temp_data.head(10)

In [ ]:
sales_data.columns

In [ ]:
sales_data.head(10)

In [ ]:
tomatoes_data = sales_data[sales_data.Vegetable=='Tomatoes']

In [ ]:
tomatoes_data.head(10)

In [ ]:
y_1 = tomatoes_data.Revenue_USD.values

In [ ]:
X_1 = temp_data.Avg_Temperature_F.values

X is an independent variable and y is the depenedant variable for the model

In [ ]:
import statsmodels.api as sm

# Add a constant (intercept term)
X_2 = sm.add_constant(X_1)

# Fit linear regression model
model = sm.OLS(y_1, X_2).fit()

# Print summary
print(model.summary())


In [ ]:
# 2024-09-01
t = 29.1
actual_sales = 1083.6

# linear model 
pred_sales = -25 * t + 2225

print(actual_sales)
print(pred_sales)

In [ ]:
# 2025-12-25
t = 15
actual_sales = '?'

# linear model 
pred_sales = -25 * t + 2225

print(actual_sales)
print(pred_sales)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. Prepare Data
# ensure float32 instead of double
X = torch.tensor(X_1, dtype=torch.float32).view(-1, 1)   # shape [N,1]
y = (-25 * X + 2225).view(-1, 1)                         # same shape as output

# 2. Define the Model
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(in_features=1, out_features=1) # default float32

    def forward(self, x):
        return self.linear(x)

model = LinearRegression()

# 3. Define Loss Function and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.9)

# 4. Training Loop
num_epochs = 5000
for epoch in range(num_epochs):
    outputs = model(X)
    loss = criterion(outputs, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 1000 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 5. Make Predictions
predicted = model(X).detach().numpy()
print("\nFirst 5 true values:", y[:5].numpy().flatten())
print("First 5 predicted values:", predicted[:5].flatten())

# Print learned parameters
print("\nLearned Weight:", model.linear.weight.item())
print("Learned Bias:", model.linear.bias.item())


In [ ]:
pred_sales = 224.11 * t + 5.05

print(pred_sales)

In [ ]:
!pip install sagemaker

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch

session = sagemaker.Session()
role = "<Your-IAM-Role-ARN>"

estimator = PyTorch(
    entry_point="train.py",
    role=role,
    framework_version="1.12",
    py_version="py38",
    instance_type="ml.m5.large",
    instance_count=1,
)

estimator.fit()


## What this code does:

1. **Creates a checkpoint directory** - Makes a folder called `model_checkpoints` to store your saved models

2. **Saves the complete checkpoint** - Saves not just the model weights, but also:
   - Model state dictionary (weights and biases)
   - Optimizer state (for resuming training)
   - Current epoch number
   - Final loss value
   - Model configuration

3. **Loads the checkpoint** - Demonstrates how to:
   - Create a new model instance
   - Load the saved weights
   - Verify the loaded model matches the original

4. **Tests the loaded model** - Makes predictions with both models to ensure they're identical

## Key benefits of this approach:

- **Resume training**: You can continue training from where you left off
- **Model sharing**: Save and share your trained model with others
- **Production deployment**: Load the trained model for inference
- **Experiment tracking**: Keep track of different model versions

You can add this code to your notebook after your training loop to save your model checkpoint!

In [ ]:
# 6. Save Model Checkpoint
print("\n=== Saving Model Checkpoint ===")

# Create checkpoint directory if it doesn't exist
import os
checkpoint_dir = "model_checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# Save the complete model checkpoint
checkpoint_path = os.path.join(checkpoint_dir, "linear_regression_checkpoint.pth")
checkpoint = {
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss.item(),
    'model_config': {
        'in_features': 1,
        'out_features': 1
    }
}

torch.save(checkpoint, checkpoint_path)
print(f"Model checkpoint saved to: {checkpoint_path}")

# 7. Load Model Checkpoint
print("\n=== Loading Model Checkpoint ===")

# Create a new model instance
loaded_model = LinearRegression()

# Load the checkpoint
loaded_checkpoint = torch.load(checkpoint_path)
loaded_model.load_state_dict(loaded_checkpoint['model_state_dict'])

# Set model to evaluation mode
loaded_model.eval()

# Verify the loaded model has the same parameters
print(f"Original model weight: {model.linear.weight.item():.4f}")
print(f"Loaded model weight: {loaded_model.linear.weight.item():.4f}")
print(f"Original model bias: {model.linear.bias.item():.4f}")
print(f"Loaded model bias: {loaded_model.linear.bias.item():.4f}")

# Test prediction with loaded model
with torch.no_grad():
    test_input = torch.tensor([[25.0]], dtype=torch.float32)  # Test temperature of 25°F
    original_prediction = model(test_input).item()
    loaded_prediction = loaded_model(test_input).item()
    
    print(f"\nTest prediction at 25°F:")
    print(f"Original model: {original_prediction:.2f}")
    print(f"Loaded model: {loaded_prediction:.2f}")

print("\nModel checkpoint successfully saved and loaded!")

I'll show you the code for model validation and performance evaluation. Here's comprehensive code to validate your model and assess its performance:

## What this validation code provides:

### 1. **Data Splitting**
- Splits data into training (80%) and validation (20%) sets
- Ensures unbiased performance evaluation

### 2. **Training with Validation**
- Monitors both training and validation loss during training
- Helps detect overfitting early

### 3. **Comprehensive Metrics**
- **MSE/RMSE**: Measures prediction error
- **MAE**: Mean absolute error (robust to outliers)
- **R² Score**: Proportion of variance explained (0-1, higher is better)

### 4. **Overfitting Detection**
- Compares training vs validation performance
- Warns about potential overfitting or underfitting

### 5. **Residual Analysis**
- Checks if predictions are unbiased
- Analyzes error distribution

### 6. **Cross-Validation**
- K-fold cross-validation for robust performance estimation
- Provides confidence intervals for performance metrics

### 7. **Baseline Comparison**
- Compares against simple baseline (mean prediction)
- Shows actual improvement over naive approaches

### 8. **Model Persistence**
- Saves the validated model with performance metrics
- Enables reproducible results

This comprehensive validation approach will give you confidence in your model's performance and help identify any issues before deployment!

In [ ]:

# Model Validation and Performance Evaluation
print("\n=== Model Validation and Performance ===")

# 1. Split data into training and validation sets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Convert PyTorch tensors to numpy for sklearn functions
X_np = X.numpy().flatten()
y_np = y.numpy().flatten()

# Split data (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(
    X_np, y_np, test_size=0.2, random_state=42
)

# Convert back to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).view(-1, 1)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).view(-1, 1)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

# 2. Train model on training data only
print("Training model on training data...")
train_model = LinearRegression()
train_criterion = nn.MSELoss()
train_optimizer = optim.Adam(train_model.parameters(), lr=0.9)

train_epochs = 3000
train_losses = []
val_losses = []

for epoch in range(train_epochs):
    # Training
    train_model.train()
    train_outputs = train_model(X_train_tensor)
    train_loss = train_criterion(train_outputs, y_train_tensor)
    
    train_optimizer.zero_grad()
    train_loss.backward()
    train_optimizer.step()
    
    # Validation
    train_model.eval()
    with torch.no_grad():
        val_outputs = train_model(X_val_tensor)
        val_loss = train_criterion(val_outputs, y_val_tensor)
    
    train_losses.append(train_loss.item())
    val_losses.append(val_loss.item())
    
    if (epoch + 1) % 500 == 0:
        print(f'Epoch [{epoch+1}/{train_epochs}], Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

# 3. Model Performance Metrics
print("\n=== Performance Metrics ===")

# Make predictions on validation set
train_model.eval()
with torch.no_grad():
    val_predictions = train_model(X_val_tensor).numpy().flatten()
    train_predictions = train_model(X_train_tensor).numpy().flatten()

# Calculate metrics
train_mse = mean_squared_error(y_train, train_predictions)
val_mse = mean_squared_error(y_val, val_predictions)
train_rmse = np.sqrt(train_mse)
val_rmse = np.sqrt(val_mse)
train_mae = mean_absolute_error(y_train, train_predictions)
val_mae = mean_absolute_error(y_val, val_predictions)
train_r2 = r2_score(y_train, train_predictions)
val_r2 = r2_score(y_val, val_predictions)

print(f"Training Set Metrics:")
print(f"  MSE: {train_mse:.2f}")
print(f"  RMSE: {train_rmse:.2f}")
print(f"  MAE: {train_mae:.2f}")
print(f"  R² Score: {train_r2:.4f}")

print(f"\nValidation Set Metrics:")
print(f"  MSE: {val_mse:.2f}")
print(f"  RMSE: {val_rmse:.2f}")
print(f"  MAE: {val_mae:.2f}")
print(f"  R² Score: {val_r2:.4f}")

# 4. Overfitting Check
print(f"\n=== Overfitting Analysis ===")
overfitting_ratio = val_mse / train_mse if train_mse > 0 else float('inf')
print(f"Validation/Training MSE Ratio: {overfitting_ratio:.2f}")

if overfitting_ratio > 1.5:
    print("⚠️  Potential overfitting detected (validation loss much higher than training loss)")
elif overfitting_ratio < 0.8:
    print("⚠️  Potential underfitting detected (validation loss much lower than training loss)")
else:
    print("✅ Good generalization (validation and training losses are similar)")

# 5. Residual Analysis
print(f"\n=== Residual Analysis ===")
residuals = y_val - val_predictions

print(f"Residual Statistics:")
print(f"  Mean: {np.mean(residuals):.2f}")
print(f"  Std: {np.std(residuals):.2f}")
print(f"  Min: {np.min(residuals):.2f}")
print(f"  Max: {np.max(residuals):.2f}")

# Check for bias in residuals
if abs(np.mean(residuals)) > 0.1 * np.std(residuals):
    print("⚠️  Residuals show potential bias (mean significantly different from 0)")
else:
    print("✅ Residuals appear unbiased")

# 6. Cross-Validation (K-Fold)
print(f"\n=== K-Fold Cross-Validation ===")
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression as SklearnLinearRegression

# Use sklearn's LinearRegression for cross-validation
sklearn_model = SklearnLinearRegression()
cv_scores = cross_val_score(sklearn_model, X_np.reshape(-1, 1), y_np, cv=5, scoring='r2')

print(f"Cross-validation R² scores: {cv_scores}")
print(f"Mean CV R²: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# 7. Model Comparison with Baseline
print(f"\n=== Baseline Comparison ===")
# Simple baseline: predict mean of training data
baseline_predictions = np.full_like(y_val, np.mean(y_train))
baseline_mse = mean_squared_error(y_val, baseline_predictions)
baseline_r2 = r2_score(y_val, baseline_predictions)

print(f"Baseline (Mean Prediction):")
print(f"  MSE: {baseline_mse:.2f}")
print(f"  R² Score: {baseline_r2:.4f}")

print(f"\nModel vs Baseline:")
print(f"  MSE Improvement: {((baseline_mse - val_mse) / baseline_mse * 100):.1f}%")
print(f"  R² Improvement: {((val_r2 - baseline_r2) / (1 - baseline_r2) * 100):.1f}%")

# 8. Save the validated model
print(f"\n=== Saving Validated Model ===")
validated_checkpoint_path = os.path.join(checkpoint_dir, "validated_linear_regression.pth")
validated_checkpoint = {
    'epoch': train_epochs,
    'model_state_dict': train_model.state_dict(),
    'optimizer_state_dict': train_optimizer.state_dict(),
    'train_loss': train_losses[-1],
    'val_loss': val_losses[-1],
    'performance_metrics': {
        'train_mse': train_mse,
        'val_mse': val_mse,
        'train_r2': train_r2,
        'val_r2': val_r2,
        'cv_mean_r2': cv_scores.mean(),
        'cv_std_r2': cv_scores.std()
    },
    'model_config': {
        'in_features': 1,
        'out_features': 1
    }
}

torch.save(validated_checkpoint, validated_checkpoint_path)
print(f"Validated model saved to: {validated_checkpoint_path}")

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np
import torch
import torch.nn as nn

app = FastAPI()

# Define the same model architecture as used in training
class LinearRegressionModel(nn.Module):
    def __init__(self, in_features=1, out_features=1):
        super().__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x):
        return self.linear(x)

# Load the trained model
MODEL_PATH = "checkpoints/validated_linear_regression.pth"
model = LinearRegressionModel()
checkpoint = torch.load(MODEL_PATH, map_location=torch.device('cpu'), weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

class InputData(BaseModel):
    x: float

class OutputData(BaseModel):
    prediction: float

@app.post("/predict", response_model=OutputData)
def predict(data: InputData):
    # Prepare input for PyTorch model
    x_tensor = torch.tensor([[data.x]], dtype=torch.float32)
    with torch.no_grad():
        y_pred = model(x_tensor)
    return OutputData(prediction=float(y_pred.item()))

# To run: uvicorn <filename>:app --reload



In [ ]:
import requests

# Sample code to test the FastAPI endpoint
def test_fastapi_predict():
    url = "http://127.0.0.1:8000/predict"
    payload = {"x": 2.5}
    response = requests.post(url, json=payload)
    if response.status_code == 200:
        print("Prediction:", response.json())
    else:
        print("Request failed with status code:", response.status_code)
        print("Response:", response.text)

# Uncomment the following line to run the test
# test_fastapi_predict()
